In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle


In [2]:
data=pd.read_csv("data/Churn_Modelling.csv")

In [3]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)


In [4]:
gender_encoder=LabelEncoder()
data['Gender']=gender_encoder.fit_transform(data['Gender'])


In [5]:
from sklearn.preprocessing import OneHotEncoder
encoder=OneHotEncoder()
geo_encoder=encoder.fit_transform(data[['Geography']]).toarray()
geo_encoder
geo_encoded_df=pd.DataFrame(geo_encoder,columns=encoder.get_feature_names_out(['Geography']))


In [6]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)


In [7]:

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(gender_encoder,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(encoder,file)


In [8]:
X=data.drop('Exited',axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [9]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping


In [11]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1,activation='sigmoid')
])

/home/chisty/new-projects/ANN model/venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-01-30 14:24:23.327338: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [ ]:
model.summary()

In [14]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [15]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [16]:
result=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=early_stopping_callback
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8040 - loss: 0.4723 - val_accuracy: 0.8390 - val_loss: 0.3834
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8430 - loss: 0.3806 - val_accuracy: 0.8560 - val_loss: 0.3516
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8510 - loss: 0.3589 - val_accuracy: 0.8600 - val_loss: 0.3429
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8611 - loss: 0.3378 - val_accuracy: 0.8620 - val_loss: 0.3457
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8556 - loss: 0.3446 - val_accuracy: 0.8620 - val_loss: 0.3429
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8688 - loss: 0.3263 - val_accuracy: 0.8550 - val_loss: 0.3445
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8650 - loss: 0.3235 - val_accuracy: 0.8600 - val_loss: 0.3444
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8694 - loss: 0.3254 - val_accu

In [18]:
model.save('model.keras')